# Notebook 3: Summarizing Energy News Articles

This notebook demonstrates how to use the `analysis` service to summarize the news articles fetched earlier.

**Objective:** Generate concise summaries of energy market news articles using a pre-trained Hugging Face model (e.g., BART).

## 1. Setup

Import necessary libraries, add the project root to the path, and load the analysis service. This will load the summarization model specified in `analysis.py` (e.g., `facebook/bart-large-cnn`). This might take time on the first run.

In [ ]:
import osimport sysimport jsonimport logging# Setup logginglogging.basicConfig(level=logging.INFO)logger = logging.getLogger(__name__)# Add the project root to the Python pathproject_root = os.path.abspath(os.path.join(os.getcwd(), '..'))if project_root not in sys.path:    sys.path.append(project_root)# Import the analysis servicetry:    from api.services import analysis    import importlib    importlib.reload(analysis) # Ensure latest code and models are loaded    models_loaded = analysis.summarizer_pipeline is not None    if models_loaded:        logger.info('Summarization model loaded successfully.')    else:        logger.warning('Summarization model failed to load. Check logs in analysis.py.')except Exception as e:    logger.error(f'Failed to import or load analysis service: {e}', exc_info=True)    models_loaded = False

## 2. Load Fetched Articles

Load the articles saved by `01_news_fetching.ipynb`.

In [ ]:
articles = []articles_file = os.path.join("data", "fetched_articles.json")if os.path.exists(articles_file):    try:        with open(articles_file, 'r') as f:            articles = json.load(f)        print(f"Loaded {len(articles)} articles from {articles_file}")    except Exception as e:        print(f"Error loading articles from {articles_file}: {e}")else:    print(f"Error: {articles_file} not found. Please run Notebook 1 first.")

## 3. Perform Summarization

Iterate through the articles and apply the `summarize_text` function.

In [ ]:
if models_loaded and articles:    print("Performing summarization...")    results = []    for i, article in enumerate(articles):        title = article.get('title', 'No Title')        content = article.get('content') or article.get('description')        print(f"\nSummarizing article {i+1}: {title}")                if content:            summary = analysis.summarize_text(content) # Use default length ratios            if summary:                print(f"  Summary: {summary}")                results.append({                    'title': title,                    'url': article.get('url'),                    'original_length': len(content.split()), # Approx word count                    'summary_length': len(summary.split()),                    'summary': summary                })            else:                print("  Summarization failed for this article.")                results.append({                    'title': title,                    'url': article.get('url'),                    'error': 'Summarization failed'                })        else:            print("  Skipping summarization: No content or description found.")            results.append({                'title': title,                'url': article.get('url'),                'error': 'No content available'            })                # Save results    results_file = os.path.join("data", "summary_results.json")    with open(results_file, 'w') as f:        json.dump(results, f, indent=2)    print(f"\nSaved summarization results to {results_file}")            elif not models_loaded:    print("Skipping summarization as the model failed to load.")elif not articles:    print("Skipping summarization as no articles were loaded.")

## 4. Conclusion

This notebook demonstrated how to generate summaries for the fetched news articles using the `analysis.py` service. The summaries have been saved to `notebooks/data/summary_results.json`.